# Projet maison n° 3

In [1]:
# imports
import glob
import os.path
import pandas as pd
from pandas.api.types import CategoricalDtype

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [2]:
# names us
def df_names_us():
    dfs=[]
    df_all = pd.DataFrame()
    
    # liste des fichiers
    files=glob.glob("names/yob*.txt")
    # triés dans l'ordre comme ça s'est fait 
    files.sort()

    for file in files:
        df = pd.read_csv(file, sep=",", header=None)
        #on renome les colonnes 
        df.rename(columns={0: 'name', 1: 'gender', 2: 'births'}, inplace=True)

        # on récup l'année 
        year = os.path.basename(file)[3:7]
        df["year"]=year
        # déplacement de la colonne year en 1er
        df.insert(0, "year", df.pop("year"))
        dfs.append(df)

    df_all = pd.concat(dfs, ignore_index=True)

    #df_all['gender'].nunique()  # 2 genders
    #df_all['year'].nunique() # 141 year (2020-1880+1)

    #reduce_size(df_all)
    df_all['year'] = df_all['year'].astype(int)

    return df_all

names_us = df_names_us()
names_us

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2020858,2020,Zykell,M,5
2020859,2020,Zylus,M,5
2020860,2020,Zymari,M,5
2020861,2020,Zyn,M,5


In [3]:
names_us.head(5)

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


In [4]:
names_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020863 entries, 0 to 2020862
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   year    int64 
 1   name    object
 2   gender  object
 3   births  int64 
dtypes: int64(2), object(2)
memory usage: 61.7+ MB


Tiens des Unknown... ?

In [5]:
# à considérer... pour suppression. Sans indication, elles restent
names_us.loc[names_us['name']=='Unknown']


,year,name,gender,births
1990,1880,Unknown,M,5
3638,1881,Unknown,M,8
5792,1882,Unknown,M,7
7907,1883,Unknown,M,7
12468,1885,Unknown,M,7
...,...,...,...,...
1945315,2018,Unknown,M,83
1961045,2019,Unknown,F,45
1977579,2019,Unknown,M,70
1993711,2020,Unknown,F,34


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [6]:
### names fr
def df_names_fr():
    dff = pd.DataFrame()

    dff = load_data(dff)
    dff = compat_gender(dff)
    dff = clean(dff)
    dff = names1(dff)
    dff = sort_n_index(dff)
    return dff

def load_data(dff):
    
    # liste des fichiers
    filename="nat2020.csv"
    dff = pd.read_csv(filename, sep=";", header=0)
    #on renome les colonnes 
    dff.rename(columns={'sexe': 'gender'}, inplace=True)
    dff.insert(0, "year", dff.pop("annais"))
    dff.insert(1, "name", dff.pop("preusuel"))
    dff.insert(3, "births", dff.pop("nombre"))
    
    #reduce_size(dff)  
    return dff

#  - Mêmes valeurs pour la colonne 'gender'
def compat_gender(dff):
    #masque = dff['gender']==1
    dff['gender'] = dff['gender'].apply(lambda x: 'M' if x == 1 else 'F')
    return dff

# crea de la df et
#  - Même ordre et mêmes noms des colonnes : year, name, gender, births


In [7]:
def clean(dff):
    #  - Mêmes dtypes pour les colonnes
    # nécessite du cleanup
    dff = dff.dropna(subset=['year'])

    # on doit supprimer les entrées avec année XXXX
    # elles sont inutiles et empechent le passage en int 
    # on conserve les entrées comportant 4 digits 'dddd'
    
    dff = dff.loc[ dff['year'].str.contains("^[0-9][0-9][0-9][0-9]$")]
    dff['year'] = dff['year'].astype(int)
    
    return dff

In [8]:
def names1(dff):
    dff["name"] = dff["name"].astype(str)
    
    # - Seuls les prénoms de 2 caractères et plus sont conservés
    # suppression des 'name' de longueur < 2
    dff = dff.loc[dff['name'].str.len() >= 2]
    
    # - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
    #dff['name'] = dff['name'].apply(lambda x: x.title())
    dff['name'] = dff['name'].apply(lambda x: x.title())
    
    # - Les lignes avec des données inutilisables doivent être supprimées
    dff = dff.loc[dff['name'] != "_Prenoms_Rares" ]

    dff.loc[dff['name'].isnull() ]
    dff = dff.dropna(subset=['name'])
    
    return dff

In [9]:
def sort_n_index(names_fr):
    # - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
    names_fr = names_fr.sort_values(by=['year', 'gender', 'births', 'name'], 
                 ascending=[True, True, False, True])


    # - L'index du DataFrame doit aller de 0 à N-1
    names_fr.index = pd.RangeIndex(len(names_fr.index))
    #names_fr.info()
    return names_fr

sans précision particlière on chosir de supprimer les entrées avec année XXXX

In [10]:
names_fr=df_names_fr()
names_fr

/tmp/ipykernel_15318/3503502974.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['name'] = dff['name'].apply(lambda x: x.title())


,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
630402,2020,Élouen,M,3
630403,2020,Énes,M,3
630404,2020,Éphraïm,M,3
630405,2020,Éros,M,3


In [11]:
names_fr.shape

(630407, 4)

On peut réduire la taille en utilisant des CategoricalDtype

Les valeurs de 'year' ne sont plus de type int dans ce cas

In [12]:
# frugalité, réduction la taille du corpus
def reduce_size(df):
    genders = ["M", "F"]
    cat_genders = CategoricalDtype(categories=genders, ordered=True)
    df["gender"] = df["gender"].astype(cat_genders)

    years = [str(i) for i in range(1880, 2021)]
    cat_y = CategoricalDtype(categories=years, ordered=True)
    df["year"] = df["year"].astype(cat_y)
    return df

In [13]:
names_us = reduce_size(names_us)
names_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2020863 entries, 0 to 2020862
Data columns (total 4 columns):
 #   Column  Dtype   
---  ------  -----   
 0   year    category
 1   name    object  
 2   gender  category
 3   births  int64   
dtypes: category(2), int64(1), object(1)
memory usage: 36.6+ MB


In [14]:
names_fr = reduce_size(names_fr)
names_fr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630407 entries, 0 to 630406
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   year    0 non-null       category
 1   name    630407 non-null  object  
 2   gender  630407 non-null  category
 3   births  630407 non-null  int64   
dtypes: category(2), int64(1), object(1)
memory usage: 11.4+ MB


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [15]:
import pandas as pd

FILENAME="Webstat_Export_20210929.csv"

import re
regex = re.compile('\((.*)\)')
def extr(txt):
    r = re.search(regex, txt)
    if r:
        return r.group(1)
    else:
        return txt

def load():
# taux de change
    dfc = pd.read_csv(FILENAME, 
                      header=0, 
                      skiprows=lambda x: x in [1, 2, 3, 4, 5], 
                      #index_col='Titre :',
                      sep=';', 
                      encoding="utf-8",
                      na_values=['-',''])
    dfc.columns = [extr(x) for x in dfc.columns]
    
    dfc = dfc.set_index('Titre :')
    dfc.index = pd.to_datetime(dfc.index,format='%d/%m/%Y')
    dfc = dfc.dropna()
    dfc = dfc.replace(',','.',regex=True).astype(float)
    return dfc

def df_taux_change(devises):
    dftx=load()
    return pd.DataFrame(dftx[devises],columns=devises)

load()

,AUD,BGN,BRL,CAD,CHF,CNY,CYP,CZK,DKK,EEK,...,RON,RUB,SEK,SGD,SIT,SKK,THB,TRY,USD,ZAR
Titre :,,,,,,,,,,,,,,,,,,,,,
2006-12-29,1.6691,1.9558,2.8141,1.5281,1.6069,10.2793,0.57820,27.485,7.4560,15.6466,...,3.3835,34.6800,9.0404,2.0202,239.6400,34.435,46.7700,1.8640,1.3170,9.2124
2006-12-28,1.6699,1.9558,2.8175,1.5268,1.6058,10.2935,0.57820,27.540,7.4573,15.6466,...,3.3980,34.6940,9.0463,2.0209,239.6400,34.561,47.2520,1.8648,1.3173,9.2590
2006-12-27,1.6765,1.9558,2.8343,1.5297,1.6058,10.2923,0.57820,27.590,7.4543,15.6466,...,3.3695,34.6660,9.0230,2.0217,239.6500,34.349,47.7760,1.8751,1.3159,9.2198
2006-12-22,1.6792,1.9558,2.8391,1.5240,1.6029,10.3113,0.57810,27.573,7.4536,15.6466,...,3.3510,34.7030,8.9888,2.0310,239.6600,34.378,48.0190,1.8810,1.3192,9.2195
2006-12-21,1.6785,1.9558,2.8460,1.5155,1.6032,10.3012,0.57810,27.525,7.4545,15.6466,...,3.3759,34.6730,8.9661,2.0303,239.6900,34.585,48.1590,1.8836,1.3178,9.2280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000-07-25,1.6072,1.9462,1.6858,1.3803,1.5538,7.7887,0.57425,35.660,7.4548,15.6466,...,2.0453,26.0328,8.3880,1.6424,207.9904,42.472,38.5104,0.5923,0.9410,6.5443
2000-07-24,1.5905,1.9464,1.6774,1.3700,1.5555,7.7241,0.57395,35.570,7.4559,15.6466,...,2.0258,25.7812,8.4220,1.6281,207.8613,42.494,38.0236,0.5889,0.9332,6.4489
2000-07-21,1.5945,1.9465,1.6821,1.3783,1.5511,7.7498,0.57399,35.595,7.4550,15.6466,...,2.0295,25.8311,8.3880,1.6342,207.8603,42.533,37.7107,0.5893,0.9363,6.4495


In [16]:

df_taux_change(['USD'])

,USD
Titre :,
2006-12-29,1.3170
2006-12-28,1.3173
2006-12-27,1.3159
2006-12-22,1.3192
2006-12-21,1.3178
...,...
2000-07-25,0.9410
2000-07-24,0.9332
2000-07-21,0.9363


In [17]:


#dftx.rename(columns={'A': 'Col_1'}

### Tests

In [18]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [19]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [20]:
# run tests
run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... /tmp/ipykernel_15318/3503502974.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['name'] = dff['name'].apply(lambda x: x.title())
ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 5.346s

OK
